<a href="https://colab.research.google.com/github/Darshan2104/Transformer-Adapter/blob/main/T5%2Badapter/T5-small%20%2B%20Adapter(CNN)_on_XLSum(eng)-test(all).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install Transformers
!pip install sentencepiece
!pip install adapter-Transformers
!pip install rouge
!pip install rouge_score

     |████████████████████████████████| 311 kB 5.6 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 1.1 MB 43.3 MB/s 
     |████████████████████████████████| 212 kB 46.0 MB/s 
     |████████████████████████████████| 134 kB 46.3 MB/s 
     |████████████████████████████████| 271 kB 49.5 MB/s 
     |████████████████████████████████| 144 kB 48.1 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 3.8 MB 5.4 MB/s 
     |████████████████████████████████| 6.5 MB 35.0 MB/s 
     |████████████████████████████████| 895 kB 40.1 MB/s 
     |████████████████████████████████| 596 kB 41.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 3.2 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 

In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, AutoTokenizer,AutoModel, T5Config, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments,Seq2SeqTrainer
import torch
from datasets import load_dataset, list_datasets,load_metric
import torch
import sentencepiece as spm
import numpy as np
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# list_datasets()

In [4]:
ds_test = load_dataset('GEM/xlsum' ,"english", split='test') 

Downloading:   0%|          | 0.00/4.82k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/83.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xlsum downloaded and prepared to /root/.cache/huggingface/datasets/GEM___xlsum/english/2.0.0/c5f94b79254b76efed292f24957fe663c4b35c83e91284fbefc51adf4aea8dc0. Subsequent calls will reuse this data.


In [5]:
ds_test

Dataset({
    features: ['gem_id', 'url', 'title', 'target', 'references', 'text'],
    num_rows: 11535
})

In [6]:
def add_prefix(example):
  example['target'] = 'summarize: ' + example['target']
  return example

ds_test = ds_test.map(add_prefix)

0ex [00:00, ?ex/s]

In [7]:
# Failed as it's not a pd it is Dataset object so 
# ds_test['article'] = 'summarize: ' + ds_test['article']
# ds_test['highlights'] = '<pad>' + ds_test['highlights']

In [8]:
print("Size of test :",len(ds_test['target']))

Size of test : 11535


In [9]:
test_texts, test_labels = ds_test['target'], ds_test['title']

In [10]:
# Creating model with adapter 
from transformers import T5Config, T5Model, AdapterConfig, AutoModelWithHeads, get_adapter_info, AutoModelForSeq2SeqLM

model = AutoModel.from_pretrained('t5-small')

# adapter

adapter_name = model.load_adapter('/content/drive/MyDrive/t5-small-train-adapter-cnn/cnn_dailymail')
model.save_pretrained('my-T5-small')

# print(model.config)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [11]:
%%time
# with adapter

model = AutoModelForSeq2SeqLM.from_pretrained('/content/my-T5-small')
model.set_active_adapters(adapter_name)

tokenizer = AutoTokenizer.from_pretrained("t5-small")
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

CPU times: user 1.32 s, sys: 271 ms, total: 1.59 s
Wall time: 3.18 s


In [12]:
# converting our encodings in to Dataset objects
class PegasusDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])  # torch.tensor(self.labels[idx])
        return item
        
    def __len__(self):
        return len(self.labels['input_ids'])

#####################################################################################################################################################
#####################################################################################################################################################

def prepare_data(train_texts, train_labels, 
                 val_texts=None, val_labels=None, 
                 test_texts=None, test_labels=None):
  """
  Prepare input data for model fine-tuning
  """

  # create tokenizer for our model
  prepare_val = False if val_texts is None or val_labels is None else True
  prepare_test = False if test_texts is None or test_labels is None else True

# create encodings from out text data

  def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True)
    decodings = tokenizer(labels, truncation=True, padding=True)
    dataset_tokenized = PegasusDataset(encodings, decodings)
    return dataset_tokenized

# create combinded dataset object from text and label encodings
  train_dataset = tokenize_data(train_texts, train_labels)
  val_dataset = tokenize_data(val_texts, val_labels) if prepare_val else None
  test_dataset = tokenize_data(test_texts, test_labels) if prepare_test else None

  return train_dataset, val_dataset, test_dataset

#####################################################################################################################################################
#####################################################################################################################################################

metric = load_metric('rouge')

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels
    
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

#####################################################################################################################################################
#####################################################################################################################################################

def prepare_fine_tuning(test_dataset, freeze_encoder=False, output_dir='./results'):
  
  """
  Prepare configurations and base model for fine-tuning
  """
  torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

  args = Seq2SeqTrainingArguments(
      output_dir=output_dir,           # output directory
      per_device_eval_batch_size=4,   # batch size for evaluation
      predict_with_generate=True,
    )

  trainer = Seq2SeqTrainer(
      model=model,                         
      args=args,                 
      compute_metrics=compute_metrics if args.predict_with_generate else None,
      tokenizer=tokenizer,
    )
  return trainer

Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

In [13]:
test_dataset,_,_ = prepare_data(test_texts, test_labels)

In [14]:
trainer = prepare_fine_tuning(test_dataset)

In [15]:
%%time
predict_results = trainer.predict(test_dataset,metric_key_prefix="predict")

***** Running Prediction *****
  Num examples = 11535
  Batch size = 4


CPU times: user 19min 4s, sys: 7.93 s, total: 19min 12s
Wall time: 19min 11s


In [16]:
predict_results.metrics

{'predict_gen_len': 17.9924,
 'predict_loss': 11.921663284301758,
 'predict_rouge1': 34.6128,
 'predict_rouge2': 13.1016,
 'predict_rougeL': 29.4834,
 'predict_rougeLsum': 29.6201,
 'predict_runtime': 1151.3351,
 'predict_samples_per_second': 10.019,
 'predict_steps_per_second': 2.505}

In [17]:
final_output = tokenizer.batch_decode(predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)

In [18]:
final_output[:2]

['Donald Trump campaigned on becoming a president unlike any Washington has ever seen. With',
 'The Welsh Government "would probably" take the UK government to court if it struck an international']

In [21]:
rouge = load_metric('rouge')
id =[] 
text=[]
predicted_summary =[]
actual_summary =[]
r1 = []
r2 = []
rL =[]
rLSum = []

for i in range(1,len(ds_test['target'])+1):
  index = i
  txt = test_texts[(i-1):(i)]
  actual = test_labels[(i-1):(i)]
  predicted = final_output[(i-1):(i)]

  result = rouge.compute(predictions=predicted, references=actual)
# print(result['rouge1'].mid.fmeasure)
  result1 = {key: value.mid.fmeasure * 100 for key, value in result.items()}
  print(f'{i} -mid : {result1}')

  id.append(index)
  text.append(txt)
  predicted_summary.append(predicted)
  actual_summary.append(actual)
  r1.append(result1['rouge1'])
  r2.append(result1['rouge2'])
  rL.append(result1['rougeL'])
  rLSum.append(result1['rougeLsum'])


# .........................................................................
# Individual score :

# from rouge import Rouge

# rouge = Rouge()
# rouge.get_scores(final_output, test_labels, avg=True)

Streaming output truncated to the last 5000 lines.
6536 -mid : {'rouge1': 44.44444444444445, 'rouge2': 16.0, 'rougeL': 44.44444444444445, 'rougeLsum': 44.44444444444445}
6537 -mid : {'rouge1': 19.04761904761905, 'rouge2': 10.526315789473685, 'rougeL': 19.04761904761905, 'rougeLsum': 19.04761904761905}
6538 -mid : {'rouge1': 34.78260869565217, 'rouge2': 9.523809523809526, 'rougeL': 17.391304347826086, 'rougeLsum': 17.391304347826086}
6539 -mid : {'rouge1': 38.0952380952381, 'rouge2': 10.526315789473685, 'rougeL': 28.57142857142857, 'rougeLsum': 28.57142857142857}
6540 -mid : {'rouge1': 32.0, 'rouge2': 8.695652173913045, 'rougeL': 24.000000000000004, 'rougeLsum': 24.000000000000004}
6541 -mid : {'rouge1': 25.0, 'rouge2': 14.285714285714288, 'rougeL': 25.0, 'rougeLsum': 25.0}
6542 -mid : {'rouge1': 24.000000000000004, 'rouge2': 8.695652173913045, 'rougeL': 16.0, 'rougeLsum': 16.0}
6543 -mid : {'rouge1': 11.76470588235294, 'rouge2': 0.0, 'rougeL': 11.76470588235294, 'rougeLsum': 11.7647058

In [23]:
import csv
from itertools import zip_longest

headline =('id', 'Summary', 'Predicted Headline','Actual Headline','rouge1','rouge2','rougeL','rougeLsum')
data = [id,text, predicted_summary,actual_summary,r1,r2,rL,rLSum]
export_data = zip_longest(*data, fillvalue = '')
with open('/content/XLsum(test)_on_T5-small+adapter_CNN.csv', 'w', newline='') as file:
      write = csv.writer(file)
      write.writerow(headline)
      write.writerows(export_data)

## **For manual Testing**

In [ ]:
# print(model.config)

In [ ]:
#Source : https://huggingface.co/docs/transformers/model_doc/t5

# inputs = tokenizer([sentence for sentence in test_texts], return_tensors="pt",truncation=True, padding=True)

In [ ]:
# labels = tokenizer([sentence for sentence in test_labels], return_tensors="pt",truncation=True, padding=True).input_ids

In [ ]:
# test_modeling_t5

In [ ]:
# loss = model(inputs,labels= labels).loss

In [ ]:
# mtf_score = -(labels.shape[-1] * loss.item())
# print(mtf_score)

In [ ]:
# model.config

In [ ]:
# inputs

In [ ]:
%%time
# output_sequences = model.generate(
#     input_ids=inputs['input_ids'],
#     num_beams=1,
# )

CPU times: user 6.92 s, sys: 314 ms, total: 7.23 s
Wall time: 7.57 s


In [ ]:
# final_output = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)

In [ ]:
# final_output

In [ ]:
# .........................................................................
# .........................................................................

rouge = load_metric('rouge')
result = rouge.compute(predictions=final_output, references=test_labels)
# print(result['rouge1'].mid.fmeasure)

result1 = {key: value.low.fmeasure * 100 for key, value in result.items()}
print(f'Low : {result1}')

result2 = {key: value.mid.fmeasure * 100 for key, value in result.items()}
print(f'Mid : {result2}')

result3 = {key: value.high.fmeasure * 100 for key, value in result.items()}
print(f'High : {result3}')

# .........................................................................
# .........................................................................
# Individual score :

# from rouge import Rouge

# rouge = Rouge()
# rouge.get_scores(final_output, test_labels, avg=True)

Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

Low : {'rouge1': 24.05341810792376, 'rouge2': 6.431451210826214, 'rougeL': 18.59177725663147, 'rougeLsum': 21.827739948026295}
Mid : {'rouge1': 30.55740839230993, 'rouge2': 13.413401739872329, 'rougeL': 25.72941215645671, 'rougeLsum': 27.89156417728317}
High : {'rouge1': 39.12707626910413, 'rouge2': 24.194693656017183, 'rougeL': 35.38447822170327, 'rougeLsum': 36.82596287224365}


## **Experimental Work-Place**

In [ ]:
name = 'Hello'

In [ ]:
result2

In [ ]:
# update dictionary
result2.update( [('Austria', 43),('Name',name)] )

In [ ]:
result2

In [ ]:
import csv

field_name = ['rouge1','rouge2','rougeL','rougeLsum']

print(type(field_name))
with open('./result.csv','a') as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames = field_name)  
  writer.writeheader()
  writer.writerows([result2])
  csvfile.close()

In [ ]:
import csv

field_name = ['r1','r2','r3']
dictionaryobj= [{'r1':'1','r3':'10','r2':'5'}]
print(type(dictionaryobj))
with open('./output.csv','a') as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames = field_name)  
  # writer.writeheader()
  writer.writerows(dictionaryobj)
  csvfile.close()

In [ ]:
import csv

field_name = ['r1','r2','r3']
dictionaryobj= [{'r1':'11','r2':'15','r3':'11'}]
print(type(dictionaryobj))
with open('./output.csv','a') as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames = field_name)  
  # writer.writeheader()
  writer.writerows(dictionaryobj)
  csvfile.close()